In [2]:
import json

file_path = '../stage1_problems/STAGE1_1.json'

with open(file_path, 'r') as file:
    data = json.load(file)



In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [4]:
class Order:
    def __init__(self, order_info):
        # [ORD_ID, ORD_TIME, SHOP_LAT, SHOP_LON, DLV_LAT, DLV_LON, VOL, COOK_TIME, DLV_DEADLINE]
        self.id = order_info[0]
        self.order_time = order_info[1]
        self.shop_lat = order_info[2]
        self.shop_lon = order_info[3]
        self.dlv_lat = order_info[4]
        self.dlv_lon = order_info[5]
        self.cook_time = order_info[6]
        self.volume = order_info[7]
        self.deadline = order_info[8]

        self.ready_time = self.order_time + self.cook_time

    def __repr__(self) -> str:
        return f'Order([{self.id}, {self.order_time}, {self.shop_lat}, {self.shop_lon}, {self.dlv_lat}, {self.dlv_lon}, {self.volume}, {self.cook_time}, {self.deadline}])'

# 배달원 class
class Rider:
    def __init__(self, rider_info):
        # [type, speed, capa, var_cost, fixed_cost, service_time, available number]
        self.type = rider_info[0]
        self.speed = rider_info[1]
        self.capa = rider_info[2]
        self.var_cost = rider_info[3]
        self.fixed_cost = rider_info[4]
        self.service_time = rider_info[5]
        self.available_number = rider_info[6]
    
    def __repr__(self) -> str:
        return f'Rider([{self.type}, {self.speed}, {self.capa}, {self.var_cost}, {self.fixed_cost}, {self.service_time}, {self.available_number}])'
    
    # 주어진 거리에 대한 배달원 비용 계산
    # = 배달원별 고정비 + 이동거리로 계산된 변동비
    def calculate_cost(self, dist):
        return self.fixed_cost + dist / 100.0 * self.var_cost

# 묶음 주문 정보
class Bundle:
    def __init__(self, all_orders, rider, shop_seq, dlv_seq, total_volume, total_dist, feasible=True):
        self.rider = rider
        self.all_orders = all_orders
        self.feasible = feasible
        self.shop_seq = shop_seq
        self.dlv_seq = dlv_seq
        self.total_volume = total_volume
        self.total_dist = total_dist
        self.update_cost()

    # 묶음 주문의 비용 update
    def update_cost(self):
        self.cost = self.rider.calculate_cost(self.total_dist)
        self.cost_per_ord = self.cost / len(self.shop_seq)


    def __repr__(self) -> str:
        return f'Bundle(all_orders, {self.rider.type}, {self.shop_seq}, {self.dlv_seq}, {self.total_volume}, {self.feasible})'


In [5]:
riders_data = data['RIDERS']
orders_data = data['ORDERS']
dist_data = data['DIST']

riders_columns = ['Transport', 'Speed', 'Capacity', 'RestTime', 'Cost', 'Income', 'Satisfaction']
riders_df = pd.DataFrame(riders_data, columns=riders_columns)

orders_columns = ['OrderID', 'Time', 'StartLat', 'StartLong', 'EndLat', 'EndLong', 'Distance', 'TimeWindow', 'deadline']
orders_df = pd.DataFrame(orders_data, columns=orders_columns)

dist_df = pd.DataFrame(dist_data)

In [6]:
riders_df

,Transport,Speed,Capacity,RestTime,Cost,Income,Satisfaction
0,BIKE,5.291005,100,60,8000,120,20
1,WALK,1.322751,70,30,8000,120,30
2,CAR,4.232804,200,100,6000,180,100


In [7]:
orders_df

,OrderID,Time,StartLat,StartLong,EndLat,EndLong,Distance,TimeWindow,deadline
0,0,28,35.997628,125.982584,35.995128,125.977870,600,21,2176
1,1,86,36.012988,126.004387,36.004457,126.004163,900,12,2680
2,2,118,35.991074,126.022580,35.984552,126.029739,600,22,2419
3,3,155,36.021098,126.019330,36.016537,126.019030,600,44,2303
4,4,156,36.018153,125.972029,36.017555,125.986989,900,16,2882
...,...,...,...,...,...,...,...,...,...
95,95,3466,35.991851,125.992472,35.988031,125.993688,600,13,5591
96,96,3474,36.016543,125.996638,36.018567,125.998289,900,28,5843
97,97,3493,35.990046,126.040079,35.988455,126.048018,600,27,5717
98,98,3496,36.001260,126.004988,35.999274,126.010367,1200,33,6252


In [8]:
import math

def haversine(lat1, lon1, lat2, lon2):
    # 지구의 반지름 (단위: km)
    radius = 6371

    # 위도와 경도를 라디안으로 변환
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # 위도와 경도의 차이 계산
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine 공식 적용
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = radius * c

    return distance

In [9]:
haversine(orders_df['StartLat'][0], orders_df['StartLong'][0], orders_df['StartLat'][1], orders_df['StartLong'][1])

2.600671621390343

In [10]:
K = data['K']

all_orders = [Order(order_info) for order_info in data['ORDERS']]
all_riders = [Rider(rider_info) for rider_info in data['RIDERS']]

dist = np.array(data['DIST'])
for r in all_riders:
    r.T = np.round(dist/r.speed + r.service_time)

In [26]:
all_riders[0].speed

5.291005291005291

In [ ]:
r_car = np.round(dist/r.speed + r.service_time)

In [21]:
r.type

'CAR'

In [11]:
data['K']

100

In [12]:
len(all_orders)

100

In [13]:
all_riders

[Rider([BIKE, 5.291005291005291, 100, 60, 8000, 120, 20]),
 Rider([WALK, 1.3227513227513228, 70, 30, 8000, 120, 30]),
 Rider([CAR, 4.2328042328042335, 200, 100, 6000, 180, 100])]

In [14]:
pd.DataFrame(dist)

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0,3645,5146,5902,3465,4475,2941,7568,6473,2709,...,1032,2524,2913,4024,2738,2049,3817,8374,3512,3228
1,3645,0,4114,2268,4158,3785,4899,5801,4478,3511,...,4184,3011,1123,2716,4490,4117,1161,6699,2266,3118
2,5146,4114,0,4697,7644,7739,3606,2471,1703,6838,...,6156,6371,4847,1440,3331,3674,5267,3234,2001,6788
3,5902,2268,4697,0,5980,5121,6721,5405,4179,5491,...,6445,5025,3252,3713,6307,6083,2681,6242,3584,4911
4,3465,4158,7644,5980,0,1546,6365,9753,8469,829,...,2804,1273,3058,6239,6107,5431,3311,10640,5658,1072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2049,4117,3674,6083,5431,6224,950,6140,5262,4630,...,3027,4336,3918,3018,689,0,4794,6851,2737,4996
196,3817,1161,5267,2681,3311,2685,5663,6934,5608,2815,...,4075,2363,918,3854,5272,4794,0,7831,3370,2239
197,8374,6699,3234,6242,10640,10483,6613,899,2224,9867,...,9377,9378,7659,4461,6416,6851,7831,0,5038,9695
198,3512,2266,2001,3584,5658,5750,3200,4177,2990,4862,...,4448,4386,2861,582,2797,2737,3370,5038,0,4787


In [19]:
3645/4.23280423 + 180

1041.1312505704996

In [16]:
pd.DataFrame(r.T)

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,180.0,1041.0,1396.0,1574.0,999.0,1237.0,875.0,1968.0,1709.0,820.0,...,424.0,776.0,868.0,1131.0,827.0,664.0,1082.0,2158.0,1010.0,943.0
1,1041.0,180.0,1152.0,716.0,1162.0,1074.0,1337.0,1550.0,1238.0,1009.0,...,1168.0,891.0,445.0,822.0,1241.0,1153.0,454.0,1763.0,715.0,917.0
2,1396.0,1152.0,180.0,1290.0,1986.0,2008.0,1032.0,764.0,582.0,1795.0,...,1634.0,1685.0,1325.0,520.0,967.0,1048.0,1424.0,944.0,653.0,1784.0
3,1574.0,716.0,1290.0,180.0,1593.0,1390.0,1768.0,1457.0,1167.0,1477.0,...,1703.0,1367.0,948.0,1057.0,1670.0,1617.0,813.0,1655.0,1027.0,1340.0
4,999.0,1162.0,1986.0,1593.0,180.0,545.0,1684.0,2484.0,2181.0,376.0,...,842.0,481.0,902.0,1654.0,1623.0,1463.0,962.0,2694.0,1517.0,433.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,664.0,1153.0,1048.0,1617.0,1463.0,1650.0,404.0,1631.0,1423.0,1274.0,...,895.0,1204.0,1106.0,893.0,343.0,180.0,1313.0,1799.0,827.0,1360.0
196,1082.0,454.0,1424.0,813.0,962.0,814.0,1518.0,1818.0,1505.0,845.0,...,1143.0,738.0,397.0,1091.0,1426.0,1313.0,180.0,2030.0,976.0,709.0
197,2158.0,1763.0,944.0,1655.0,2694.0,2657.0,1742.0,392.0,705.0,2511.0,...,2395.0,2396.0,1989.0,1234.0,1696.0,1799.0,2030.0,180.0,1370.0,2470.0
198,1010.0,715.0,653.0,1027.0,1517.0,1538.0,936.0,1167.0,886.0,1329.0,...,1231.0,1216.0,856.0,317.0,841.0,827.0,976.0,1370.0,180.0,1311.0


In [18]:
np.diag(r.T)

array([180., 180., 180., 180., 180., 180., 180., 180., 180., 180., 180.,
       180., 180., 180., 180., 180., 180., 180., 180., 180., 180., 180.,
       180., 180., 180., 180., 180., 180., 180., 180., 180., 180., 180.,
       180., 180., 180., 180., 180., 180., 180., 180., 180., 180., 180.,
       180., 180., 180., 180., 180., 180., 180., 180., 180., 180., 180.,
       180., 180., 180., 180., 180., 180., 180., 180., 180., 180., 180.,
       180., 180., 180., 180., 180., 180., 180., 180., 180., 180., 180.,
       180., 180., 180., 180., 180., 180., 180., 180., 180., 180., 180.,
       180., 180., 180., 180., 180., 180., 180., 180., 180., 180., 180.,
       180., 180., 180., 180., 180., 180., 180., 180., 180., 180., 180.,
       180., 180., 180., 180., 180., 180., 180., 180., 180., 180., 180.,
       180., 180., 180., 180., 180., 180., 180., 180., 180., 180., 180.,
       180., 180., 180., 180., 180., 180., 180., 180., 180., 180., 180.,
       180., 180., 180., 180., 180., 180., 180., 18